In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hydroeval import evaluator, nse, kge, pbias
from tqdm import tqdm
import datetime as dt

In [3]:
df = pd.read_csv("../data/pnwNP_modelStats_DELETEME.csv")

In [5]:
df[df["kge_b"]<0]

,Unnamed: 0,gage,model,nse,kge,kge_r,kge_a,kge_b,n_wateryear,site_no,dec_lat_va,dec_long_va,numZero,numOne,startDate,endDate,comid,numObs,propZero,propOne
1718,1718,14202650,nwm0,-0.053110,-0.931900,-0.202744,0.041595,-0.169232,2,14202650,45.440642,-123.125000,1,140,2010-04-14,2020-12-31,23805138.0,3914,0.000255,0.035769
1719,1719,14202650,nwm1,-0.038002,-0.871674,-0.141334,0.021714,-0.115113,2,14202650,45.440642,-123.125000,1,140,2010-04-14,2020-12-31,23805138.0,3914,0.000255,0.035769
1774,1774,14211820,nwm0,-0.000643,-1.450993,-0.006270,0.007307,-1.002335,27,14211820,45.639005,-122.763155,2,1343,1988-10-15,2020-12-31,23815084.0,11765,0.000170,0.114152
1775,1775,14211820,nwm1,-0.000340,-1.038187,-0.011464,0.003074,-0.461946,33,14211820,45.639005,-122.763155,2,1343,1988-10-15,2020-12-31,23815084.0,11765,0.000170,0.114152


In [32]:
evaluator(kge,
                simulations = dat['streamflow_NWM2d0'],
                evaluation = dat["streamflow_NWIS"], 
                axis=1,
                transform='inv',
                epsilon=0.00001)

array([[-0.72647433, -0.01571435,  0.00172426,  0.02404738]])

In [6]:
dat = pd.read_csv("../data/pnwNP_modeledData/14202650.csv")

In [10]:
dat.describe()

,Unnamed: 0,streamflow_VIC,streamflow_PRMS,streamflow_NWM2d0,streamflow_NWM2d1,streamflow_NWIS,gage
count,15310.000000,0.0,0.0,9496.000000,15310.000000,533.000000,15310.0
mean,7654.500000,NaN,NaN,1.428885,1.284433,1.049491,14202650.0
std,4419.760646,NaN,NaN,2.226126,2.111928,1.411188,0.0
min,0.000000,NaN,NaN,0.020000,0.040000,-2.132259,14202650.0
25%,3827.250000,NaN,NaN,0.110000,0.150000,0.020388,14202650.0
50%,7654.500000,NaN,NaN,0.654792,0.481667,0.521030,14202650.0
75%,11481.750000,NaN,NaN,1.709167,1.509062,1.614060,14202650.0
max,15309.000000,NaN,NaN,34.875416,38.670832,7.022578,14202650.0


In [22]:
((1/(np.mean(dat["streamflow_NWM2d0"])+0.01))/(1/np.mean(dat["streamflow_NWIS"]))-1)

0.7218023798195474

In [31]:
0.04*0.01

0.0004